# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [13]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


Делаем индексом "Показатель":

In [14]:
df1 = df.set_index('Показатель')
df1


,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


Преобразуем нормативное описание к однотипному виду. (В данном случае беру математические символы неравенства).

In [15]:

df1['Норматив'] = df1['Норматив'].replace(to_replace=r'(не более (\d-)?)|-', value='<', regex=True)
df1['Норматив'] = df1['Норматив'].replace(to_replace=r'в пределах ', value='', regex=True)
df1['Норматив'] = df1['Норматив'].replace(to_replace=r',', value='.', regex=True)
df1['Результат анализа'] = df1['Результат анализа'].replace(to_replace=r'б/цвета', value='0', regex=True)
df1

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,6<9
Запах,баллы,1,<3
Цветность,градусы,0,<30
Жёсткость,мг-эквл/дм3,9.199999999999999,7<10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,<1.5
Нитриты (по NO2),мг/дм3,0.017,<3.3
Нитраты (по NO3),мг/дм3,24,<45
Фосфаты (P),мг/дм3,0.36,<3.5
Хлориды (Cl),мг/дм3,200,<350


Разделяю диапазоны значений нормативов, добавляю колонки минимальных и максимальных знаений. В значения "не более", минимальным считаем ноль. 


In [16]:
df2 = df1['Норматив'].str.split('<',expand=True)
df2.rename(columns = ({0:'Минимум', 1:'Максимум'}), inplace=True)
df_result = pd.concat([df1, df2], axis=1)
del df_result['Норматив']
df_result['Минимум'] = df_result['Минимум'].replace(to_replace='', value='0', regex=True)
df_result[['Результат анализа', 'Максимум', 'Минимум']] = df_result[['Результат анализа', 'Максимум','Минимум']].astype(float)
df_result


,Единица измерений,Результат анализа,Минимум,Максимум
Показатель,,,,
pH,единицы pH,8.400,6.0,9.0
Запах,баллы,1.000,0.0,3.0
Цветность,градусы,0.000,0.0,30.0
Жёсткость,мг-эквл/дм3,9.200,7.0,10.0
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,0.0,1.5
Нитриты (по NO2),мг/дм3,0.017,0.0,3.3
Нитраты (по NO3),мг/дм3,24.000,0.0,45.0
Фосфаты (P),мг/дм3,0.360,0.0,3.5
Хлориды (Cl),мг/дм3,200.000,0.0,350.0


In [17]:
df_result = df_result.assign(Сравнение_с_min=df_result['Минимум'] - df_result['Результат анализа'])
df_result = df_result.assign(Сравнение_с_max=df_result['Результат анализа'] - df_result['Максимум'])
df_result

,Единица измерений,Результат анализа,Минимум,Максимум,Сравнение_с_min,Сравнение_с_max
Показатель,,,,,,
pH,единицы pH,8.400,6.0,9.0,-2.400,-0.600
Запах,баллы,1.000,0.0,3.0,-1.000,-2.000
Цветность,градусы,0.000,0.0,30.0,0.000,-30.000
Жёсткость,мг-эквл/дм3,9.200,7.0,10.0,-2.200,-0.800
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,0.0,1.5,-0.420,-1.080
Нитриты (по NO2),мг/дм3,0.017,0.0,3.3,-0.017,-3.283
Нитраты (по NO3),мг/дм3,24.000,0.0,45.0,-24.000,-21.000
Фосфаты (P),мг/дм3,0.360,0.0,3.5,-0.360,-3.140
Хлориды (Cl),мг/дм3,200.000,0.0,350.0,-200.000,-150.000


Добавляем колонки 'Сравнение_с_min' и 'Сравнение_с_max' и вычисляем значение. Положительное значение будет говорить нам о нарушении нормативов.

Делаем выводы по каждому показателю:

In [18]:
for index, row in df_result.iterrows():
    message = 'Для показателя "' + index + '" действует норматив от '+ str(row['Минимум']) + ' до ' + str(row['Максимум']) + '. Единицы измерения: ' + str(row['Единица измерений']) 
    if row['Сравнение_с_min'] <= 0 and row['Сравнение_с_max'] <= 0:
        message1 = '. Превышение не выявлено.'
    elif row['Сравнение_с_min'] >= 0:
        message1 = '. Норматив нарушен. Показатель ниже минимального порогового значения на ' + str(row['Сравнение_с_min']) + str(row['Единица измерений']) 
    elif row['Сравнение_с_max'] >= 0:
        message1 = '. Норматив нарушен. Показатель выше максимального порогового значения на ' + str(row['Сравнение_с_max']) + str(row['Единица измерений']) 
    message_result= message + message1
    print(message_result)

Для показателя "pH" действует норматив от 6.0 до 9.0. Единицы измерения: единицы pH. Превышение не выявлено.
Для показателя "Запах" действует норматив от 0.0 до 3.0. Единицы измерения: баллы. Превышение не выявлено.
Для показателя "Цветность" действует норматив от 0.0 до 30.0. Единицы измерения: градусы. Превышение не выявлено.
Для показателя "Жёсткость" действует норматив от 7.0 до 10.0. Единицы измерения: мг-эквл/дм3. Превышение не выявлено.
Для показателя "Аммиак и аммоний-ион (по азоту)" действует норматив от 0.0 до 1.5. Единицы измерения: мг/дм3. Превышение не выявлено.
Для показателя "Нитриты (по NO2)" действует норматив от 0.0 до 3.3. Единицы измерения: мг/дм3. Превышение не выявлено.
Для показателя "Нитраты (по NO3)" действует норматив от 0.0 до 45.0. Единицы измерения: мг/дм3. Превышение не выявлено.
Для показателя "Фосфаты (P)" действует норматив от 0.0 до 3.5. Единицы измерения: мг/дм3. Превышение не выявлено.
Для показателя "Хлориды (Cl)" действует норматив от 0.0 до 350.0.

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [19]:
def fruit(orange, apple, quantity):
    sum_fruit = orange + apple
    probability = orange/sum_fruit
    i = 1
    while i < quantity:
        orange -= 1
        sum_fruit -= 1
        probability = probability * (orange/sum_fruit)
        i += 1
    return probability
        

print(fruit(5, 4, 3))

0.11904761904761904


Да, ответ 0,119 подтверждается

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

Очевидно, что вероятность события А1 равна = 3/10,  кроме того, P = 7/9 , так как перед взятием второй детали у мастера осталось 9 деталей, из которых только 2 нестандартные и 7 стандартных. По теореме умножения
P = 3/10 * 7/9 = 7/30
0.2333

In [26]:
result_search = []
number_experiments = 100001
def search_details(non_standard, num_details, number_checks):
    standard = num_details - non_standard
    for number_experiment in range(1, number_experiments):
            temp_list = []
            details = ['standard'] * standard + ['non_standard'] * non_standard
            for _ in range (number_checks):
                detail = random.choice(details)
                temp_list.append(detail)
                details.remove(detail)
            result_search.append(temp_list)


    df = pd.DataFrame(result_search) 
        
    probability = len(df[(df[0] == 'non_standard') & (df[1] == 'standard')]) / number_experiments

    return f'Вероятность того, что {number_checks}-я деталь стандартная, составляет {round(probability, 3)}.'

print(search_details(3, 10, 2))


Вероятность того, что 2-я деталь стандартная, составляет 0.233.


Да, ответ подтверждается экспериментально